# 💻 Б7: Старт/стоп кодони во произволен CDS

```{admonition} Опис на барање
:class: tip
Одберете еден CDS и најдете ги сите старт и стоп кодони во
неговата РНА.
```

За произволен CDS врз кој ќе експериментирам, се одлучив да одам со "<code>pim</code>" генот, ;**YP_pPCP05**:

In [1]:
from Bio import SeqIO
gene_record = SeqIO.read("yersinia-pestis-fasta/NC_005816.gb", "genbank")

print(gene_record.features[21])

type: CDS
location: [4342:4780](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GI:45478716', 'GeneID:2767712']
    Key: gene, Value: ['pim']
    Key: locus_tag, Value: ['YP_pPCP05']
    Key: note, Value: ['similar to many previously sequenced pesticin immunity protein entries of Yersinia pestis plasmid pPCP, e.g. gi| 16082683|,ref|NP_395230.1| (NC_003132) , gi|1200166|emb|CAA90861.1| (Z54145 ) , gi|1488655| emb|CAA63439.1| (X92856) , gi|2996219|gb|AAC62543.1| (AF053945) , and gi|5763814|emb|CAB531 67.1| (AL109969)']
    Key: product, Value: ['pesticin immunity protein']
    Key: protein_id, Value: ['NP_995571.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MGGGMISKLFCLALIFLSSSGLAEKNTYTAKDILQNLELNTFGNSLSHGIYGKQTTFKQTEFTNIKSNTKKHIALINKDNSWMISLKILGIKRDEYTVCFEDFSLIRPPTYVAIHPLLIKKVKSGNFIVVKEIKKSIPGCTVYYH']



Локацијата ја зимам од главната секвенца и истотака табелата за преведување на RNA е со вредност 11! Ова е доста корисно за транслација.

Зимање на подстринг од главната секвенца:

In [2]:
sub_record = gene_record[4342:4780]
print(sub_record)
print(len(sub_record))

ID: NC_005816.1
Name: NC_005816
Description: Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence
Number of features: 2
Seq('ATGGGAGGGGGAATGATCTCAAAGTTATTTTGCTTGGCTCTCATATTTTTATCA...TAA')
438


Добивање на **RNA** со транскрипција од подстрингот:

In [3]:
sub_rna = sub_record.seq.transcribe()
print(sub_rna)

AUGGGAGGGGGAAUGAUCUCAAAGUUAUUUUGCUUGGCUCUCAUAUUUUUAUCAUCAAGUGGCCUUGCAGAAAAAAACACAUAUACAGCAAAAGACAUCUUGCAAAACCUAGAAUUAAAUACCUUUGGCAAUUCAUUGUCUCAUGGCAUCUAUGGGAAACAGACAACCUUCAAGCAAACCGAGUUUACAAAUAUUAAAAGCAACACCAAAAAACACAUUGCACUUAUCAAUAAAGACAACUCAUGGAUGAUAUCAUUAAAAAUACUAGGAAUUAAGAGAGAUGAGUAUACUGUCUGUUUUGAAGAUUUCUCUCUAAUAAGACCGCCAACAUAUGUAGCCAUACAUCCUCUACUUAUAAAAAAAGUAAAAUCUGGAAACUUUAUAGUAGUGAAAGAAAUAAAGAAAUCUAUCCCUGGUUGCACUGUAUAUUAUCAUUAA


Табела за процесот на транслација:

In [6]:
from Bio.Data import CodonTable

mito_table = CodonTable.unambiguous_rna_by_id[11]
print(mito_table)

Table 11 Bacterial, Archaeal, Plant Plastid

  |  U      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
U | UUU F   | UCU S   | UAU Y   | UGU C   | U
U | UUC F   | UCC S   | UAC Y   | UGC C   | C
U | UUA L   | UCA S   | UAA Stop| UGA Stop| A
U | UUG L(s)| UCG S   | UAG Stop| UGG W   | G
--+---------+---------+---------+---------+--
C | CUU L   | CCU P   | CAU H   | CGU R   | U
C | CUC L   | CCC P   | CAC H   | CGC R   | C
C | CUA L   | CCA P   | CAA Q   | CGA R   | A
C | CUG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | AUU I(s)| ACU T   | AAU N   | AGU S   | U
A | AUC I(s)| ACC T   | AAC N   | AGC S   | C
A | AUA I(s)| ACA T   | AAA K   | AGA R   | A
A | AUG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GUU V   | GCU A   | GAU D   | GGU G   | U
G | GUC V   | GCC A   | GAC D   | GGC G   | C
G | GUA V   | GCA A   | GAA E   | GGA G   | A
G | GUG V(s)| GCG A   | GAG E   | GGG

Соодветните старт/стоп кодони според табелата со број 11:

In [7]:
print(f"Старт кодони: {mito_table.start_codons}")
print(f"Стоп кодони: {mito_table.stop_codons}")

Старт кодони: ['UUG', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUG']
Стоп кодони: ['UAA', 'UAG', 'UGA']


In [8]:
# Листи за позиции на старт и стоп кодони
start_codon_positions = []
stop_codon_positions = []
# Листи за имињата на најдените кодони
start_found = []
stop_found = []
# Конверзија во стринг, за полесна работа
rna_string = str(sub_rna)

"""
Идеја на пребарување: 
- Започни од 0 и движи се до целата секвенца -2
- Земи го моменталниот триплет 
- Провери дали е во старт кодоните -> Ако е зачувај торка (позиција, триплет)
- Ако не, провери дали е во стоп кодоните -> Ако е зачувај торка (позиција, триплет)
- Зголеми го бројачот 
"""

k = 0
while k < len(rna_string)-2:
    # extract a three-nucleotide subsequence
    current_codon = rna_string[k:k+3]
    if current_codon in mito_table.start_codons:
        start_found.append((k, current_codon))
    elif current_codon in mito_table.stop_codons:
        stop_found.append((k, current_codon))
    k += 1

print(gene_record.features[21])
print(f"Вкупна должина на CBS: {len(sub_rna)}")
print(sub_rna)
print()
print(f'Вкупен број на старт кодони: {len(start_found)}')
print(f'Најден старт кодон со позиција: {start_found}')
print()
print(f'Вкупен број на стоп кодони: {len(stop_found)}')
print(f'Најден стоп кодон со позиција: {stop_found}')

type: CDS
location: [4342:4780](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GI:45478716', 'GeneID:2767712']
    Key: gene, Value: ['pim']
    Key: locus_tag, Value: ['YP_pPCP05']
    Key: note, Value: ['similar to many previously sequenced pesticin immunity protein entries of Yersinia pestis plasmid pPCP, e.g. gi| 16082683|,ref|NP_395230.1| (NC_003132) , gi|1200166|emb|CAA90861.1| (Z54145 ) , gi|1488655| emb|CAA63439.1| (X92856) , gi|2996219|gb|AAC62543.1| (AF053945) , and gi|5763814|emb|CAB531 67.1| (AL109969)']
    Key: product, Value: ['pesticin immunity protein']
    Key: protein_id, Value: ['NP_995571.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MGGGMISKLFCLALIFLSSSGLAEKNTYTAKDILQNLELNTFGNSLSHGIYGKQTTFKQTEFTNIKSNTKKHIALINKDNSWMISLKILGIKRDEYTVCFEDFSLIRPPTYVAIHPLLIKKVKSGNFIVVKEIKKSIPGCTVYYH']

Вкупна должина на CBS: 438
AUGGGAGGGGGAAUGAUCUCAAAGUUAUUUUGCUUGGCUCUCAUAUUUUUAUCAUCAAGUGGCCUUGCAGAAAAAAACACAUAUACAGCAAAAGACAUCUUGCAA

Ова е конечниот резултат од пребарувањето и интересното што може да се забележи е дека некои од кодоните старт и стоп се поклопуваат и во следното барање при процесот на **транслација** можеме да видиме дека може произволно да зимаме комбинации од еден старт и стоп за да читаме триплети во процесот на транслација од РНА во протеинска секвенца.  